In [ ]:
#package installs

In [1]:
#Global variables
import socket
ENV_HOSTNAME = socket.gethostname()
print('ENV_HOSTNAME:' + ENV_HOSTNAME)

#store defaults for Jacob here:
ENV_FOLDER_DATA = ''
ENV_PRATT = ''


if ENV_HOSTNAME == 'JWGamingPC':
    ENV_FOLDER_DATA = 'E:\\W4732 Computer Vision\\Final Paper Data\\'
    ENV_PRATT = ''

print('ENV_FOLDER_DATA:' + ENV_FOLDER_DATA)
print('ENV_PRATT:' + ENV_PRATT)

JWGamingPC


In [ ]:
# Take initial video
# using test video #568 Dr. Rhonda Patrick
# https://archive.org/download/jre-001-837/JRE_001-837/
# https://archive.org/download/jre-001-837/JRE_001-837/Joe%20Rogan%20Experience%20%23568%20-%20Dr.%20Rhonda%20Patrick.mp4

